Kensuke Ohtake @Shinshu University

Simulation Code for "Continuous space core-periphery model with transport costs in differentiated agriculture" <a href="https://arxiv.org/abs/2206.01040" target="_blank" rel="noopener noreferrer">https://arxiv.org/abs/2206.01040</a>


In [ ]:
using Random
using Distributions
using Plots
using Dates
using Format
using DataFrames, CSV

function RK_trapez(pls)

    SD = Dates.millisecond(now())
    Random.seed!(SD) # Set seed
    
    # control parameters 
    eta = pls[1] # elasticity of substitution of aguricultural goods
    sig = pls[2] # elasticity of substitution of manufactured goods
    ta = pls[3] # agriculture transportation cost parameter
    tm = pls[4] # manufacturing transportation cost parameter
    
    function dist(z)
        minimum([abs(z), 2.0*π - abs(z)])
    end
    
    I = 128 # Number of space fragments
    dx = 2.0 * π / convert(Float64, I) # size of space fragments
    dt = 0.01 # size of time step
    mu = 0.5 # importance of manufacturing goods
    eps = 1e-10 # error
    v = 1.0 # adjustment speed of workers' migration
    
    x = zeros(Float64, I, 1)
    for i in 1:I
        x[i] = -π + (i-1) * dx
    end
    
    # Agriculture workers
    ph = ones((I, 1)) / (2.0 * π)
    
    # Transport matrix
    Exa = zeros((I, I))
    Exm = zeros((I, I))
    for i in 1:I
        for j in 1:I
            Exa[i,j] = exp(- ta * (eta - 1.0) * dist(x[i]-x[j])) # for agriculture
            Exm[i,j] = exp(- tm * (sig - 1.0) * dist(x[i]-x[j])) # for manufacturing
        end
    end
    
    # initial values of manufacturing workers
    turb_lm = rand(Uniform(-0.0005, 0.0005), (I, 1))
    turb_lm = turb_lm .- (sum(turb_lm)*dx)/(2.0*π) # total 0
    lm = (ones((I, 1)) / (2.0*π)) + turb_lm
    
    if  sum(sign.(lm)) != I
        println("Non-negativity_is_not_satisfied_!!")
    else
        ini_lm = copy(lm)
        println("Initial total manufacturing workers is ", sum(ini_lm)*dx)
    end
    
    # operator for instantaneous equilibrium
    function InstEquil(A_nominalwage_eta, M_nominalwage_sig, agr_workers, manu_workers)# Instantaneous equilibrium operator
        # A_nominalwage_eta == wA^eta
        # M_nominalwage_sig == wM^sigma
        # agr_workers = phi
        # manu_workers = lambda
        Y = mu * (M_nominalwage_sig.^(1.0/sig)) .* manu_workers + (1.0 - mu) * (A_nominalwage_eta.^(1.0/eta)) .* agr_workers
        GA_integral = Exa * (agr_workers.*(A_nominalwage_eta.^((1.0-eta)/eta)))
        GM_integral = Exm * (manu_workers.*(M_nominalwage_sig.^((1.0-sig)/sig)))
        output_1 = Exa * (Y ./ GA_integral)
        output_2 = Exm * (Y ./ GM_integral)
        return output_1, output_2
    end
    
    # Right hand side of evolution equation
    function RHS(candidate_WA, candidate_WM, agr_workers, manu_workers)
        
        # candidate_WA: candidate of fixed point of instantaneous equilibrium
        # candidate_WM: candidate of fixed point of instantaneous equilibrium
    
        # inputs must be numpy array of which shape is (I, 1)
        
        # solve fixed point problem for instantaneous equilibrium
        A_nominalwage_eta = copy(candidate_WA)
        M_nominalwage_sig = copy(candidate_WM)
        while true
            old_A_nominalwage_eta = copy(A_nominalwage_eta)
            old_M_nominalwage_sig = copy(M_nominalwage_sig)
            A_nominalwage_eta, M_nominalwage_sig = InstEquil(old_A_nominalwage_eta, old_M_nominalwage_sig, agr_workers, manu_workers)
            
            if maximum(abs.(A_nominalwage_eta - old_A_nominalwage_eta)) < eps && maximum(abs.(M_nominalwage_sig - old_M_nominalwage_sig)) < eps
                break
            end
        end
    
        GA = ((Exa * (agr_workers .* (A_nominalwage_eta .^ ((1.0-eta)/eta)))) * dx) .^ (1.0/(1.0-eta)) # price index of manufactured goods
        GM = ((Exm * (manu_workers .* (M_nominalwage_sig .^ ((1.0-sig)/sig)))) * dx) .^ (1.0/(1.0-sig)) # price index of agricultural goods
    
        rw = (M_nominalwage_sig .^ (1.0/sig)) .* (GM .^ (-mu)) .* (GA .^ (mu-1.0)) # real wage
    
        # Right hand side of evolution equation
        rhs = v * ((rw .- (transpose(rw) * manu_workers)*dx) .* manu_workers)
    
        return rhs, A_nominalwage_eta, M_nominalwage_sig
    end
    
    # time evolution  
    t = 0
    WA = ones((I, 1)) # initial candidate fixed point of instantaneous equilibrium
    WM = ones((I, 1)) # initial candidate fixed point of instantaneous equilibrium
    
    dlm = zeros((I, 1))
    while true
    
        lm_old = copy(lm)
        WA_old = copy(WA)
        WM_old = copy(WM)
        
        # 1-th order Runge Kutta method for the replicator equation 
        k_1, WA, WM = RHS(WA_old, WM_old, ph, lm)
        dlm = dt * k_1
        lm = lm_old + dlm # Euler
        
        # check nonnegativity
        if sum(sign.(lm)) != I
            println("Non-negativity_is_not_satisfied_!!")
            break
        end
        
        # check conservation of population
        if 1.0 - 1e-10 > sum(lm)*dx
            println("conservation_error_!!")
            break
        elseif sum(lm)*dx > 1.0 + 1e-10
            println("conservation_error_!!")
            break
        end
        
        if maximum(abs.(dlm)) < 1e-10
            break
        end
    
        t += 1
    
    end

    # stable or unstable
    if sum(abs.(ini_lm .- (1.0/(2.0*π)))) > sum(abs.(lm .- (1.0/(2.0*π))))
        s_or_u = "stable" # stable
    elseif sum(abs.(ini_lm .- (1.0/(2.0*π)))) < sum(abs.(lm .- (1.0/(2.0*π))))
        s_or_u = "unstable" # unstable
    else
        s_or_u = "not" # not determinable
    end
    
    # obtain stationary solutions
    stationary_WA = copy(WA)
    stationary_WM = copy(WM)
    while true
        old_stationary_WA = copy(stationary_WA)
        old_stationary_WM = copy(stationary_WM)
        stationary_WA, stationary_WM = InstEquil(old_stationary_WA, old_stationary_WM, ph, lm)
        if maximum(abs.(stationary_WA - old_stationary_WA)) < eps && maximum(abs.(stationary_WM - old_stationary_WM)) < eps
            break
        end
    end
    
    nominal_wage_A = stationary_WA .^ (1.0/eta)
    nominal_wage_M = stationary_WM .^ (1.0/sig)
    price_index_A = ((Exa * (ph .* (nominal_wage_A .^ (1.0-eta)))) * dx) .^ (1.0/(1.0-eta))
    price_index_M = ((Exm * (lm .* (nominal_wage_M .^ (1.0-sig)))) * dx) .^ (1.0/(1.0-sig))
    real_wage_A = nominal_wage_A .* (price_index_M .^ (-mu)) .* (price_index_A .^ (mu-1.0))
    real_wage_M = nominal_wage_M .* (price_index_M .^ (-mu)) .* (price_index_A .^ (mu-1.0))

    println("Final total manufacturing workers is ", sum(lm)*dx)
    
    # Plot
    DateTime = Dates.format(now(), "yyyymmddHHMMSS")# get datetime
    plt_x = append!(reshape(x, I), π)
    plt_ini_lm = append!(reshape(ini_lm, I), ini_lm[1])
    plt_lm = append!(reshape(lm, I), lm[1])
    plt_nominal_wage_A = append!(reshape(nominal_wage_A, I), nominal_wage_A[1])
    plt_nominal_wage_M = append!(reshape(nominal_wage_M, I), nominal_wage_M[1])
    plt_price_index_A = append!(reshape(price_index_A, I), price_index_A[1])
    plt_price_index_M = append!(reshape(price_index_M, I), price_index_M[1])
    plt_real_wage_A = append!(reshape(real_wage_A, I), real_wage_A[1])
    plt_real_wage_M = append!(reshape(real_wage_M, I), real_wage_M[1])
    
    filename_lm = format("{}-pop_{}_ta{}tm{}sigma{}eta{}mu{}I{}t{}sd{}", DateTime, s_or_u, ta, tm, sig, eta, mu, I, t, SD)
    newfilename_lm = replace(filename_lm, "."=>"p")

    if s_or_u == "stable"
        plot(plt_x, plt_ini_lm, title="population", label="ini_lm", dpi=300)
        plot!(plt_x, plt_lm, label="lm", dpi=300)
        savefig(newfilename_lm * ".png")
    else
        plot(plt_x, plt_lm, title="population", label="", dpi=300)
        savefig(newfilename_lm * ".png")
    end
    
    filename_rw_A = format("{}-rwA_{}_ta{}tm{}sigma{}eta{}mu{}I{}t{}sd{}", DateTime, s_or_u, ta, tm, sig, eta, mu, I, t, SD)
    newfilename_rw_A = replace(filename_rw_A, "."=>"p")
    
    plot(plt_x, plt_real_wage_A, title="agr_realwage", label="", dpi=300)
    savefig(newfilename_rw_A * ".png")
    
    filename_rw_M = format("{}-rwM_{}_ta{}tm{}sigma{}eta{}mu{}I{}t{}sd{}", DateTime, s_or_u, ta, tm, sig, eta, mu, I, t, SD)
    newfilename_rw_M = replace(filename_rw_M, "."=>"p")
    
    plot(plt_x, plt_real_wage_M, title="man_realwage", label="", dpi=300)
    savefig(newfilename_rw_M * ".png")
    
    # Output and save data as csv
    df = DataFrame(coordinates = plt_x,
                         initial_pop_share_lm = plt_ini_lm,
                         final_pop_share_lam = plt_lm,
                         nominal_wage_A = plt_nominal_wage_A,
                         nominal_wage_M = plt_nominal_wage_M,
                         price_index_A = plt_price_index_A,
                         price_index_M = plt_price_index_M,
                         real_wage_A = plt_real_wage_A,
                         real_wage_M = plt_real_wage_M) # Save data as dataframe
    #df_ind = df.set_index('coordinates') # Make coodinates-column index
    filename = format("{}_ta{}tm{}sigma{}eta{}mu{}I{}sd{}", s_or_u, ta, tm, sig, eta, mu, I, SD)
    newfilename = replace(filename, "."=>"p")
    CSV.write(newfilename * ".csv", df)# Write into csv
end


In [ ]:
# Loop
for tauM in [6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.8, 0.6, 0.4, 0.2, 0.1]
    params = [2.0, 3.0, 6.0, tauM] # set parameters [eta, sigma, tau_A, tau_M]
    for itr in 1:5
        RK_trapez(params)
    end
end
